In [5]:
import re
import requests
import csv
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os, glob
import time
import pandas as pd
import numpy as np
    
# Tole dvoje je treba, da se Chrome sproti ne odpira
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)

C:\Users\jasak\AppData\Local\Temp\ipykernel_20420\159072978.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)


In [6]:
vzorec_bloka = re.compile(
    r'<td><mtg-set-icon.*?'
    r'</tr',
    flags=re.DOTALL
)

vzorec_karte = re.compile(
    r'<td><mtg-set-icon.*?>.*?<i _ngcontent.*? class="ss ss-fw ss-(?P<set>.+?)\b( ss-.*?)?"></i><!----></mtg-set-icon>'
    r'<a href="/prints/(?P<id_karte>\d*).*?">'
    r'(?P<ime>.*?)'
    r'</a>.*?</td>.*?<td> (?P<redkost>.*?) </td>'
    r'.*?<td class="text-end">'
    r' (?P<povprecna_cena>.*?) </td>(<\!---->)+(<td class="text-end">.*?</td>.*?<td class="text-end"> (?P<povprecna_cena_foil>.*?) </td>.*?<td class="text-end">.*?</td>)?',  
    flags=re.DOTALL
    )

def izloci_podatke_o_kartah(blok):
    karta = vzorec_karte.search(blok).groupdict()
    karta['id_karte'] = int(karta['id_karte'])
    karta['ime'] = karta['ime'].strip()
    karta['set'] = karta['set'].upper()
    karta['redkost'] = karta["redkost"].strip()
    karta['povprecna_cena'] = karta["povprecna_cena"]
    karta['povprecna_cena_foil'] = karta["povprecna_cena_foil"]

    return karta

def izloci_podatke_o_specificni_karti(vsebina):
    """
    To bo izločilo vse dodatne podatke o karti, ki jih bomo našli na naslovu .../prints/<id_karte>
    """
    pass

def pridobi_ustrezno_ime_lokalne_datoteke(st_strani):
    return f"Podatki o kartah/Podatki iz setov/Karte iz seta st. {st_strani}.html"

In [ ]:
# Najprej poradiramo csv in json datoteki
with open("karte.csv", "w") as dat:
    pass

with open("karte.json", "w") as dat:
    pass

PRVI_SET = 1
ZADNJI_SET = 1300

#indeksi_setov_ki_obstajajo = []

for st_strani in range(PRVI_SET, ZADNJI_SET + 1):
    url = (
        f'https://www.mtgstocks.com/sets/{st_strani}' 
    )
    print(f"\n Zajemam {url}\n")
    driver.get(url)
    time.sleep(1)
    
    
    vsebina = driver.page_source
    naslov_strani = driver.title
    
    stevilo_podstrani_s_kartami = vsebina.count("pagination-page page-item")
    st_besed_Market_na_strani = vsebina.count(">Market<")
    
    print("Naslov strani:", naslov_strani)
    
    if naslov_strani in ["File not found - MTGStocks"]:
        print(f'\n Stran št. {st_strani} ni dosegljiva.\n')
    else:
        print(f'\n Stran št. {st_strani} JE dosegljiva.\n')
        if "Art Series:" in naslov_strani or "From the Vault:" in naslov_strani or st_besed_Market_na_strani == 0:
            continue
        else:
            with open(pridobi_ustrezno_ime_lokalne_datoteke(st_strani), 'w') as dat:
                driver.find_element(by=By.XPATH, value = "//*[text() = 'Market']//..").click()
                for podstran in range(1, stevilo_podstrani_s_kartami + 1):
                    driver.find_element(by=By.XPATH, value = f"//*[contains(@class, 'pagination-page page-item')][{podstran}]//a").click()
                    time.sleep(1)
                    print(f"Pobrana {podstran}. podstran")
                    dat.write(driver.page_source)
  

Koda za zapisovanje kart iz lokalnih datotek v csv in json format.


In [9]:
debug_mode_za_branje_iz_lokalnih_datotek = False
path = 'Podatki o kartah/Podatki iz setov'
karte = []

for filename in glob.glob(os.path.join(path, '*.html')):
    with open(filename, 'r') as f:

        stetje = 0
        print(f"Berem podatke kart iz {filename}")
        vsebina = f.read()
        
        for blok in vzorec_bloka.finditer(vsebina):
            stetje += 1
            
            if debug_mode_za_branje_iz_lokalnih_datotek:
                print(stetje)
                print(blok.group(0))
                print(vzorec_karte.search(blok.group(0)).groupdict())
            
            karte.append(izloci_podatke_o_kartah(blok.group(0)))

with open("karte.json", "a") as dat:
    json.dump(karte, dat, indent=4, ensure_ascii=False)
    

with open("karte.csv", "a") as dat:
    writer = csv.DictWriter(dat, [
        "id_karte",
        "ime",
        "set",
        "redkost",
        "povprecna_cena",
        "povprecna_cena_foil",
    ])
    writer.writeheader()
    writer.writerows(karte)

Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 10.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 100.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1008.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1009.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 101.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1011.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 102.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 103.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 104.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1041.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 105.html
Ber

Zdaj se znebimo kart, ki so tokeni ali basic landi
Prav tako bi se radi znebili kart, ki imajo tako ceno kot ceno foila ničelno. (Torej da lahko karte primerjamo po vsaj eni komponenti)
Pri tem je vredno omeniti, da imajo karte, ki so najbolj zaželene, tudi najbolj popolne podatke, torej jih s tem ne bomo izgubili


In [10]:
podatki_o_kartah = pd.read_csv("karte.csv", encoding="latin1")

regex_izraz_za_basic_lande = r'Forest \(\d|Mountain \(\d|Swamp \(\d|Plains \(\d|Island \(\d|Wastes \(\d'
okrajsani_podatki = podatki_o_kartah[~(podatki_o_kartah.redkost.isin(["Token", "Land"])) & ~(podatki_o_kartah.ime.str.match(regex_izraz_za_basic_lande)) & ~(podatki_o_kartah.ime.isin(["Forest", "Mountain", "Swamp", "Plains", "Island", "Wastes"]))]
okrajsani_podatki = okrajsani_podatki[~(okrajsani_podatki.povprecna_cena.isnull() & okrajsani_podatki.povprecna_cena_foil.isnull())]
okrajsani_indeksi = okrajsani_podatki.id_karte
print(okrajsani_podatki)
print(okrajsani_indeksi)

      id_karte               ime  set   redkost povprecna_cena  \
0        19748     Cyclonic Rift  RTR      Rare         $38.00   
1        19934   Utvara Hellkite  RTR    Mythic         $19.53   
2          360       Blood Crypt  RTR      Rare         $19.99   
3          404       Steam Vents  RTR      Rare         $18.63   
4          395    Overgrown Tomb  RTR      Rare         $13.99   
...        ...               ...  ...       ...            ...   
95446    18337  Dregscape Zombie  ARC    Common          $0.25   
95447    18380        Juggernaut  ARC  Uncommon          $0.25   
95448    18421           Plummet  ARC    Common          $0.25   
95449    18348          Fertilid  ARC    Common          $0.38   
95450    18434  Sanctum Gargoyle  ARC    Common          $0.24   

      povprecna_cena_foil  
0                  $57.99  
1                  $57.44  
2                  $34.37  
3                  $38.61  
4                  $25.72  
...                   ...  
95446      

Zdaj pa pogledamo vsako posamezno karto


In [ ]:
print("Število vseh indeksov kart, ki jih imamo:", len(okrajsani_indeksi))
for indeks_printa in okrajsani_indeksi[:10]:
    url = f"https://www.mtgstocks.com/prints/{indeks_printa}"
    print(f"Zajemanje printa karte št. {indeks_printa} s strani {url}")
    with open(f"Podatki o kartah/Podatki o specificnih kartah/Podatki o karti st. {indeks_printa}.html", "w", encoding="utf-8") as dat:
        try:
            driver.get(url)
            time.sleep(1)
            dat.write(driver.page_source)
        except:
            pass
        